# Testing Adversarial Search Algorithms By Playing Othello

#### Team Members: 
Adan Soto, Jack Fitzgerald

## Abstract

Throughout the semester the board game Othello was repeatedly mentioned and discussed so this sparked a high interest within our group to develop a bot that could play the game. Later on, a competition was introduced and as a team, we were interested in participating but given some encountered bugs and other setbacks we were not able to complete our bot in time. However, even though we were not able to participate, we were still able to have many unforeseen findings and made several riveting observations based on the behavior of our bot in the many games that it played. This notebook will provide detailed information on the algorithms and techniques we researched and found the most useful while developing and testing the bot. 

## Introduction

Initially, our team planned on finding the most efficient algorithm that could sustain a high winning rate against opponents but coming to a concrete conclusion was no easy feat since there were many factors that could affect an algorithm’s success. Our interest in participating in the competition and being part of the discussion in the Slack channel allowed us to have a better and higher understanding of where and how to get started on our project. This sense of direction, as well as important details brought up by our classmates, directed our attention to extremely important issues and details we had not considered and we were able to gain some great experience. Examples of useful experience were implementing a game engine as well as a tweaking said engine to work with a basic user interface that could display games as well as the moves that had been taken throughout the game. The algorithms we researched were Minimax, Alpha-beta pruning, Montecarlo, and we later implemented a random function that served as our default function to test the algorithms against. There is already present research done on these algorithms and their ability to win games of Othello, however, many of these researchers used really detailed specifications that led us to believe we could still implement them and learn various new things from them and our own research. 

## Methods

Aside from just looking at algorithms and trying to find the most efficient one, there were other steps that had to be taken in order to be able to get to testing and or make the testing more efficient and easier to analyze results. The first step to initialize this process of implementing algorithms was to implement the game engine that was jointly developed by students of our class and review our code to make sure that it followed the requirements everyone settled on. This meant implementing GameEngine.py which provided a structure we were able to follow to develop our bot. It was clear that having some sort of user interface would be useful for testing as well as being able to observe the moves our bot was deciding on with more ease. Vue was the tool of choice to design the user interface and it worked by using JSON output to populate the game board and statistics table. However, the way the engine was originally creating the JSON out had to be implemented in order to be able to properly be used by the user interface.

Once a game engine and user interface in place, it allowed to more easily observe if the algorithms being used were working as planned or there were bugs present. As expected, plenty of bugs were present initially with moves being randomly chosen, invalid moves, or even our code forfeiting a game by timing out or not choosing any moves although moves were still possible. The minimax algorithm was implemented with limited depth and it was also randomized if it found there was an equal amount of paths available for exploration. The second algorithm used was Alpha-beta pruning which just like minimax also used a limited depth as well as the alpha-beta cutoff search from the aima-python repository. In order to make this work, a class had to be created. This class implemented necessary functions from the Game interface and was later used to allow alpha-beta to send its move to the game engine. Lastly, the Monte Carlo (UCT) algorithm was implemented, we used the modified Monte Carlo tree search which is also available in the aima-python repository.


### OthelloEngine.py

In [1]:
import time
import json
import sys
import copy


class GameEngine:
    # white_team_file will be the file name of the white team's AI file
    # black_team_file will be the file name of the black team's AI file
    # output_file is the name of the file the game will be recorded to
    # nxn is the size of the board. Default is 8x8
    # time_limit is the amount of time for each turn in seconds
    def __init__(self, white_team_bot, black_team_bot, n=8, time_limit=5.0):
        self.n = n
        self.time_limit = time_limit

        # game_state is an nxn array containing all of the positions on the board.
        #       'W', 'B' and '-' are the possible values.
        # all_moves is a list of every move in the game (for output file). Example entry for move ('W', (1, 5)):
        #        { "turn": 0,
        #          "player": "W",
        #          "time":  1.4,
        #          "move": [1, 5]
        #        }
        # turn_number track the current turn number
        # turn_times keys each team's color character ('W' or 'B') to a list of their turn times
        # total time is the current sum of all of the player's turns
        self.all_moves = []
        self.turn_number = 0
        self.turn_times = {'W': [], 'B': []}
        self.total_time = 0
        self.game_state = [['-' for i in range(n)] for j in range(n)]
        
        self.white_team = white_team_bot
        self.black_team = black_team_bot
        # Add the initial tokens to the board (class names will all be Othello_AI)
        self.game_state[n // 2 - 1][n // 2 - 1] = "W"
        self.game_state[n // 2][n // 2] = "W"
        self.game_state[n // 2 - 1][n // 2] = "B"
        self.game_state[n // 2][n // 2 - 1] = "B"
        # call play_game (returns winner)
        self.winner = self.play_game()

    # Makes all of the general calls to play the game
    def play_game(self):
        # Loop through all the moves, calling .get_move(board_state) for each team.
        #       .get_move returns a move. Example move: ('B', (4, 5))
        #       Make sure that the bot doesn't throw an error (try-except) and doesn't exceed
        #       self.time_limit
        # Call check_valid on each move. If an AI returns an invalid move, the game is over.
        # Call the update board method with the move
        # Call the check end condition on board
        # remember to add each move to all_moves
        # Also track individual turn times as well as total time
        # Return the winner: 'W', 'B', or 'T'

        # Sanity check for self.check_end()
        while True:
            # Each team takes their turn
            for team in (self.black_team, self.white_team):
                move = self.record_turn(team)
                # record_turn returns a character IFF the team's move
                # causes them to lose automatically
                if type(move) != tuple:
                    return move

                # Else update the board and check if the game is over
                else:
                    self.update_board(move)
                    self.all_moves.append(move)

                gameEnd = self.check_end();
                if gameEnd != None:
                    return gameEnd

                # Increment turn
                # Odd turns white, even turns black
                self.turn_number += 1

    # Abstract turn taking
    def record_turn(self, team):
        try:
            start = time.time()
            move = team.get_move(copy.deepcopy(self.game_state), self.turn_number)
            turnTime = time.time() - start

            if turnTime > self.time_limit:
                print("Team {} exceeded their time limit: {}".format(team.team_type, turnTime))
                raise Exception("Team {} exceeded their time limit: {}".format(team.team_type, turnTime))
            elif not self.check_valid(move):
                print("Team {} made an invalid move: {}".format(team.team_type, move))
                raise Exception("Team {} made an invalid move: {}".format(team.team_type, move))

            # Time keeping
            self.turn_times[team.team_type].append(turnTime)

            self.total_time += turnTime

            return move

        # Instant loss for the current team
        # if their turn exceeds the time limit
        # or their move is not valid
        # or their class raises an exception
        except:
            return 'B' if team.team_type == 'W' else 'W'

    # Check valid move method
    def check_valid(self, move):
        # move format: ('B', (i, j)) or ('B', None)
        # check if the given move is valid for board_state
        # A move of None indicates the player is skipping their turn. This is only valid if
        #    there are no legal moves for this player/
        # You will want to use get_all_moves
        # return True if the move is legal, and False otherwise
        current_team = 'W' if self.turn_number % 2 == 1 else 'B'
        if (move[0] != current_team):
            return False

        vMoves = get_all_moves(self.game_state, move[0])
        if move[1] is None:
            if len(vMoves) is 0:
                return True
            else:
                return False
        else:
            if move in vMoves:
                return True
            else:
                return False
        pass

    # Perform move
    def update_board(self, move):
        # move format: ('B', (i, j)) or ('B', None)
        # update the board state given the current move
        # if the move is None, do nothing
        # Assume that is a valid move, no need for extra error checking
        if move[1] is not None:
            r = move[1][0]
            c = move[1][1]
            color = move[0]

            # left
            i = r
            j = c - 1
            while j >= 0:
                if self.game_state[i][j] != color and self.game_state[i][j] != '-':
                    # it's opposite color, keep checking
                    j -= 1
                else:
                    if self.game_state[i][j] == color:
                        # it's the same color, go back and change till we are at c-1
                        for index in range(c - j - 1):
                            self.game_state[i][j + index + 1] = color
                    # end the loop
                    break

            # left-up direction
            i = r - 1
            j = c - 1
            while i >= 0 and j >= 0:
                if self.game_state[i][j] != color and self.game_state[i][j] != '-':
                    # it's opposite color, keep checking
                    i -= 1
                    j -= 1
                else:
                    if self.game_state[i][j] == color:
                        # it's the same color, go back and change till we are at c-1, r-1
                        for index in range(c - j - 1):
                            self.game_state[i + index + 1][j + index + 1] = color
                    # end the loop
                    break

            # up
            i = r - 1
            j = c
            while i >= 0:
                if self.game_state[i][j] != color and self.game_state[i][j] != '-':
                    # it's opposite color, keep checking
                    i -= 1
                else:
                    if self.game_state[i][j] == color:
                        # it's the same color, go back and change till we are at r-1
                        for index in range(r - i - 1):
                            self.game_state[i + index + 1][j] = color
                    # end the loop
                    break

            # right-up direction
            i = r - 1
            j = c + 1
            while i >= 0 and j < len(self.game_state):
                if self.game_state[i][j] != color and self.game_state[i][j] != '-':
                    # it's opposite color, keep checking
                    i -= 1
                    j += 1
                else:
                    if self.game_state[i][j] == color:
                        # it's the same color, go back and change till we are at r-1, c+1
                        for index in range(r - i - 1):
                            self.game_state[i + index + 1][j - index - 1] = color
                    # end loop
                    break

            # right direction
            i = r
            j = c + 1
            while j < len(self.game_state):
                if self.game_state[i][j] != color and self.game_state[i][j] != '-':
                    # it's opposite color, keep checking
                    j += 1
                else:
                    if self.game_state[i][j] == color:
                        # it's the same color, go back and change till we are at c+1
                        for index in range(j - c - 1):
                            self.game_state[i][j - index - 1] = color
                    # end loop
                    break

            # right-down
            i = r + 1
            j = c + 1
            while i < len(self.game_state) and j < len(self.game_state):
                if self.game_state[i][j] != color and self.game_state[i][j] != '-':
                    # it's opposite color, keep checking
                    i += 1
                    j += 1
                else:
                    if self.game_state[i][j] == color:
                        # it's the same color, go back and change till we are at r+1,c+1
                        for index in range(j - c - 1):
                            self.game_state[i - index - 1][j - index - 1] = color
                    # end loop
                    break

            # down
            i = r + 1
            j = c
            while i < len(self.game_state):
                if self.game_state[i][j] != color and self.game_state[i][j] != '-':
                    # it's opposite color, keep checking
                    i += 1
                else:
                    if self.game_state[i][j] == color:
                        # it's the same color, go back and change till we are at r+1
                        for index in range(i - r - 1):
                            self.game_state[i - index - 1][j] = color
                    # end loop
                    break

            # left-down
            i = r + 1
            j = c - 1
            while i < len(self.game_state) and j >= 0:
                if self.game_state[i][j] != color and self.game_state[i][j] != '-':
                    # it's opposite color, keep checking
                    i += 1
                    j -= 1
                else:
                    if self.game_state[i][j] == color:
                        # it's the same color, go back and change till we are at r+1
                        for index in range(i - r - 1):
                            self.game_state[i - index - 1][j + index + 1] = color
                    # end loop
                    break

            # set the spot in the game_state
            self.game_state[r][c] = color


    # Check for end condition
    def check_end(self):
        # Check the board to see if the game can continue
        # If the game is over return the winner: 'W', 'B', or 'T'
        # Otherwise, return None

        if len(get_all_moves(self.game_state, 'W')) != 0 or len(get_all_moves(self.game_state, 'B')) != 0:
            return None

        white_count = sum(row.count('W') for row in self.game_state)
        black_count = sum(row.count('B') for row in self.game_state)

        if white_count == black_count:
            return 'T'
        elif white_count > black_count:
            return 'W'
        else:
            return 'B'

    # write to output file
    # winner should be either 'W' or 'B' or 'T'
    def output_game(self, winner):
        # write a game file to self.output_file
        # See the example json formatted file for details
        # Recall that all_moves will contain a list of every move in the game

        turns = []
        for i in range(len(self.all_moves)):
            player = self.all_moves[i][0]
            turn_index = i // 2 if player == 'B' else (i - 1) // 2
            turn = {"turn": i, "player": player, "time": self.turn_times[player][turn_index],
                    "move": self.all_moves[i][1]}
            turns.append(turn)

        white_count = sum(row.count('W') for row in self.game_state)
        black_count = sum(row.count('B') for row in self.game_state)
        game_statistics = {"numTurns": len(turns), "numBlack": black_count, "numWhite":
            white_count}

        game_metadata = {"teamWhite": self.white_team.get_team_name(),
                         "teamBlack": self.black_team.get_team_name(), "winner": self.winner,
                         "statistics": game_statistics,
                         "totalTime": self.total_time}

        return json.loads(json.dumps(game_metadata))



def get_all_moves(board_state, player):
    # Return a list of all possible moves for the given player ('W' or 'B')
    # Example return value: [('W', (2, 5)), ('W', (6, 4)), ... ]
    moves = []
    adjacencies = get_adjacencies()
    for x in range(len(board_state)):
        for y in range(len(board_state[x])):
            for adjacency in adjacencies:
                if (is_valid_move(x, y, adjacency[0], adjacency[1], board_state, player, False)):
                    moves.append((player, (x, y)))
                    break
    return moves


def get_adjacencies():
    adjacencies = []
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            if (dx == 0 and dy == 0):
                continue
            adjacencies.append((dx, dy))
    return adjacencies


def is_valid_move(x, y, dx, dy, board_state, player, surrounds):
    if (board_state[x][y] != '-' and not surrounds):
        return False
    newx = x + dx
    newy = y + dy
    board_size = len(board_state)
    if (player == 'B'):
        enemy = 'W'
    else:
        enemy = 'B'
    if (newx < 0 or newx >= board_size or newy < 0 or newy >= board_size):
        return False
    newcolor = board_state[newx][newy]
    if (newcolor == enemy):
        return is_valid_move(newx, newy, dx, dy, board_state, player, True)
    if (newcolor == player):
        return surrounds
    return False

### algorithms.py

In [2]:
from math import inf
import copy
import random
import numpy as np


#____________________DEFAULT - RANDOM CHOICE____________________
def random_search(state, game):
    moves = get_all_moves(state[1], game.team_type)
    if len(moves) > 0:
        return random.choice(moves)
    return (game.team_type, None)


#____________________MINIMAX____________________
def minimax_cutoff(state, game, d=4, cutoff_test=None, eval_fn=None):
    minPlayer = ''
    maxPlayer = game.to_move(state)
    if maxPlayer == 'W':
        minPlayer = 'B'
    else:
        minPlayer = 'W'

    def max_value(state, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, maxPlayer)
        v = -inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), depth + 1))
        return v

    def min_value(state, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, maxPlayer)
        v = inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), depth + 1))
        return v

    # Body of minmax_decision:
    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state, player: game.utility(state, player))
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a), 1))


#____________________ALPHA-BETA____________________
def alpha_beta_cutoff_search(state, game, d=4, cutoff_test=None, eval_fn=None):
    minPlayer = ''
    maxPlayer = game.to_move(state)
    if maxPlayer == 'W':
        minPlayer = 'B'
    else:
        minPlayer = 'W'

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, maxPlayer)
        v = -inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(copy.deepcopy(state), a), alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, maxPlayer)
        v = inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(copy.deepcopy(state), a), alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_cutoff_search starts here:
    # The default test cuts off at depth d or at a terminal state
    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state, player: game.utility(state, player))
    best_score = -inf
    beta = inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(copy.deepcopy(state), a), best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action


#____________________MONTE CARLO - (UCT)____________________
class MCT_Node:
    """Node in the Monte Carlo search tree, keeps track of the children states."""

    def __init__(self, parent=None, state=None, U=0, N=0):
        self.__dict__.update(parent=parent, state=state, U=U, N=N)
        self.children = {}
        self.actions = None


def ucb(n, C=1.4):
    return np.inf if n.N == 0 else n.U / n.N + C * np.sqrt(np.log(n.parent.N) / n.N)

# Monte Carlo search
def monte_carlo_tree_search(state, game, N=100):
    def select(n):
        """select a leaf node in the tree"""
        if n.children:
            return select(max(n.children.keys(), key=ucb))
        else:
            return n

    def expand(n):
        """expand the leaf node by adding all its children states"""
        if not n.children and not game.terminal_test(n.state):
            n.children = {MCT_Node(state=game.result(copy.deepcopy(n.state), action), parent=n): action
                          for action in game.actions(n.state)}
        return select(n)

    def simulate(game, state):
        """simulate the utility of current state by random picking a step"""
        player = game.to_move(state)
        while not game.terminal_test(state):
            action = random.choice(list(game.actions(state)))
            state = game.result(copy.deepcopy(state), action)
        v = game.utility(state, player)
        return -v

    def backprop(n, utility):
        """passing the utility back to all parent nodes"""
        if utility > 0:
            n.U += utility
        # if utility == 0:
        #     n.U += 0.5
        n.N += 1
        if n.parent:
            backprop(n.parent, -utility)

    root = MCT_Node(state=state)

    for _ in range(N):
        leaf = select(root)
        child = expand(leaf)
        result = simulate(game, child.state)
        backprop(child, result)

    max_state = max(root.children, key=lambda p: p.N)

    return root.children.get(max_state)


### Othello_interface.py

In [3]:
from OthelloEngine import get_all_moves
import random

class Othello_AI:
    def __init__(self, team_type, adv_search_function, team_name, board_size=8, time_limit=2.0):
        # team_type will be either 'W' or 'B', indicating what color you are
        # board_size and time_limit will likely stay constant, but if you want this can add different challanges
        self.team_type = team_type
        self.adv_search_function = adv_search_function
        self.team_name = team_name

    def get_move(self, board_state, turn_number):
        # board state will be an board_size by board_size array with the current state of the game.
        #       Possible values are: 'W', 'B', or '-'
        # Return your desired move (If invalid, instant loss)
        # Example move: ('W', (1, 6))
        new_board_state = [turn_number, board_state]
        best_move = self.adv_search_function(new_board_state, self)
        return best_move

    def actions(self, board_state):
        player = self.to_move(board_state)
        moves = get_all_moves(board_state[1], player)
        if len(moves) == 0:
            return [(player, None)]
        else:
            return moves


    def result(self, board_state, move):
        if move[1] is not None:
            r = move[1][0]
            c = move[1][1]
            color = move[0]

            # left
            i = r
            j = c - 1
            while j >= 0:
                if board_state[1][i][j] != color and board_state[1][i][j] != '-':
                    # it's opposite color, keep checking
                    j -= 1
                else:
                    if board_state[1][i][j] == color:
                        # it's the same color, go back and change till we are at c-1
                        for index in range(c - j - 1):
                            board_state[1][i][j + index + 1] = color
                    # end the loop
                    break

            # left-up direction
            i = r - 1
            j = c - 1
            while i >= 0 and j >= 0:
                if board_state[1][i][j] != color and board_state[1][i][j] != '-':
                    # it's opposite color, keep checking
                    i -= 1
                    j -= 1
                else:
                    if board_state[1][i][j] == color:
                        # it's the same color, go back and change till we are at c-1, r-1
                        for index in range(c - j - 1):
                            board_state[1][i + index + 1][j + index + 1] = color
                    # end the loop
                    break

            # up
            i = r - 1
            j = c
            while i >= 0:
                if board_state[1][i][j] != color and board_state[1][i][j] != '-':
                    # it's opposite color, keep checking
                    i -= 1
                else:
                    if board_state[1][i][j] == color:
                        # it's the same color, go back and change till we are at r-1
                        for index in range(r - i - 1):
                            board_state[1][i + index + 1][j] = color
                    # end the loop
                    break

            # right-up direction
            i = r - 1
            j = c + 1
            while i >= 0 and j < len(board_state[1]):
                if board_state[1][i][j] != color and board_state[1][i][j] != '-':
                    # it's opposite color, keep checking
                    i -= 1
                    j += 1
                else:
                    if board_state[1][i][j] == color:
                        # it's the same color, go back and change till we are at r-1, c+1
                        for index in range(r - i - 1):
                            board_state[1][i + index + 1][j - index - 1] = color
                    # end loop
                    break

            # right direction
            i = r
            j = c + 1
            while j < len(board_state[1]):
                if board_state[1][i][j] != color and board_state[1][i][j] != '-':
                    # it's opposite color, keep checking
                    j += 1
                else:
                    if board_state[1][i][j] == color:
                        # it's the same color, go back and change till we are at c+1
                        for index in range(j - c - 1):
                            board_state[1][i][j - index - 1] = color
                    # end loop
                    break

            # right-down
            i = r + 1
            j = c + 1
            while i < len(board_state[1]) and j < len(board_state[1]):
                if board_state[1][i][j] != color and board_state[1][i][j] != '-':
                    # it's opposite color, keep checking
                    i += 1
                    j += 1
                else:
                    if board_state[1][i][j] == color:
                        # it's the same color, go back and change till we are at r+1,c+1
                        for index in range(j - c - 1):
                            board_state[1][i - index - 1][j - index - 1] = color
                    # end loop
                    break

            # down
            i = r + 1
            j = c
            while i < len(board_state[1]):
                if board_state[1][i][j] != color and board_state[1][i][j] != '-':
                    # it's opposite color, keep checking
                    i += 1
                else:
                    if board_state[1][i][j] == color:
                        # it's the same color, go back and change till we are at r+1
                        for index in range(i - r - 1):
                            board_state[1][i - index - 1][j] = color
                    # end loop
                    break

            # left-down
            i = r + 1
            j = c - 1
            while i < len(board_state[1]) and j >= 0:
                if board_state[1][i][j] != color and board_state[1][i][j] != '-':
                    # it's opposite color, keep checking
                    i += 1
                    j -= 1
                else:
                    if board_state[1][i][j] == color:
                        # it's the same color, go back and change till we are at r+1
                        for index in range(i - r - 1):
                            board_state[1][i - index - 1][j + index + 1] = color
                    # end loop
                    break

            # set the spot in the board_state
            board_state[1][r][c] = color
        board_state[0] += 1
        return board_state


    def terminal_test(self, board_state):
        if len(get_all_moves(board_state[1], 'W')) != 0 or len(get_all_moves(board_state[1], 'B')) != 0:
            return False
        else:
            return True


    def to_move(self, board_state):
        if board_state[0] % 2 == 1:
            return 'W'
        else:
            return 'B'


    def utility(self, board_state, player):
        # The idea behind this utility is that your total pieces compared to the opponents total pieces
        # determines who wins and by how much at the end of a game. Subtracting your opponents pieces from yours
        # will determine how much your opponent owes you (if you win), or how much you owe your opponent (if they win).

        white_count = sum(row.count('W') for row in board_state[1])
        black_count = sum(row.count('B') for row in board_state[1])

        if player == 'W':
            teamCount = white_count
            opponentCount = black_count
        else:
            teamCount = black_count
            opponentCount = white_count
        return teamCount - opponentCount


    def get_team_name(self):
        # returns a string containing your team name
        return self.team_name


## Testing

In [4]:
def test(botW, botB):    
    winList = []
    timeList = []
    
    for i in range(10):
        game = GameEngine(botW, botB)
        gameOutput = game.output_game(game.winner)
        winner = gameOutput['winner']
        totalTime = gameOutput['totalTime']
        numTurns = gameOutput['statistics']['numTurns']
        numWhite = gameOutput['statistics']['numWhite']
        numBlack = gameOutput['statistics']['numBlack']
        winList.append(winner)
        timeList.append(totalTime)
        print(" Game " + str(i) + "- winner: " + gameOutput['winner'] + " totalTime: " + str(totalTime) + " numTurns: " + str(numTurns) + " numWhite: " + str(numWhite) + " numBlack: " + str(numBlack))
        
    whiteWins = 0
    blackWins = 0
    for win in winList:
        if win == 'W':
            whiteWins += 1
        elif win == 'B':
            blackWins += 1
                
    tieGames = 10 - (whiteWins + blackWins)
                
    totalTime = 0.0
    for time in timeList:
        totalTime += time
    averageTime = totalTime / 10
    
    print(" Statistics - Average game time: " + str(averageTime) + " " + botW.get_team_name() + " Wins: " + str(whiteWins) + " " + botB.get_team_name() + " Wins: " + str(blackWins) + " Ties: " + str(tieGames))

if __name__ == "__main__":
    randomBot = Othello_AI('B', random_search, "Random bot")
    minimaxBot = Othello_AI('W', minimax_cutoff, "Minimax bot")
    alphaBetaBot = Othello_AI('W', alpha_beta_cutoff_search, "Alpha-beta bot")
    whiteMonteCarloBot = Othello_AI('W', monte_carlo_tree_search, "Monte Carlo bot")
    blackMonteCarloBot = Othello_AI('B', monte_carlo_tree_search, "Monte Carlo bot")
    
    
    # MINIMAX vs RANDOM
    print("Minimax (W) vs Random (B):")
    test(minimaxBot, randomBot)
    
    # ALPHA-BETA vs RANDOM
    print("Alpha-beta (W) vs Random (B):")
    test(alphaBetaBot, randomBot)
    
    # MONTE CARLO vs RANDOM
    print("Monte Carlo (W) vs Random (B):")
    test(whiteMonteCarloBot, randomBot)
    
    # MINIMAX vs MONTE CARLO
    print("Minimax (W) vs Monte Carlo (B):")
    test(minimaxBot, blackMonteCarloBot)
    
    # ALPHA-BETA vs MONTE CARLO
    print("Alpha-beta (W) vs Monte Carlo (B)")
    test(alphaBetaBot, blackMonteCarloBot)
    

Minimax (W) vs Random (B):
 Game 0- winner: B totalTime: 0.5597696304321289 numTurns: 60 numWhite: 21 numBlack: 43
 Game 1- winner: B totalTime: 0.6140542030334473 numTurns: 60 numWhite: 20 numBlack: 44
 Game 2- winner: B totalTime: 0.5351259708404541 numTurns: 61 numWhite: 17 numBlack: 47
 Game 3- winner: W totalTime: 0.5553598403930664 numTurns: 61 numWhite: 42 numBlack: 22
 Game 4- winner: W totalTime: 0.5740189552307129 numTurns: 60 numWhite: 46 numBlack: 18
 Game 5- winner: W totalTime: 0.5677340030670166 numTurns: 62 numWhite: 39 numBlack: 25
 Game 6- winner: B totalTime: 0.5461702346801758 numTurns: 61 numWhite: 16 numBlack: 48
 Game 7- winner: B totalTime: 0.5329880714416504 numTurns: 60 numWhite: 25 numBlack: 39
 Game 8- winner: W totalTime: 0.538982629776001 numTurns: 60 numWhite: 36 numBlack: 28
 Game 9- winner: B totalTime: 0.5343232154846191 numTurns: 60 numWhite: 19 numBlack: 45
 Statistics - Average game time: 0.5558526754379273 Minimax bot Wins: 4 Random bot Wins: 6 Tie

## Results



## Discussion

#### Minimax, Alpha-beta, Monte Carlo vs Random

#### Minimax, Alpha-beta vs Monte Carlo

#### Why no Minimax vs Alpha-beta?

An interesting phenomenon occured when we played the Minimax bot against the Alpha-beta bot, they would always play the same game. The only way to get them to play different games was to change the search depths of the algorithms, or to change the evaluation function used. This would also happen when the Minimax and Alpha-beta bots would play themselves, so we had to use the Random and Monte Carlo bots to play against the Minimax and Alpha-beta bots so that the games would be different and provide more interesting results.

The reason why Minimax and Alpha-beta keep playing the same games over and over might be because they always will pick the same path (unless the depths or evaluation functions are changed) and there isn't any randomness to the algorithms. In an attempt to randomize the alorithms, the way the actions were selected in the algorithms was randomized. However, this had no effect on the outcomes and they still had the problem of playing the same game. It is currently unclear to us why this behavior is actually occurring.

## Conclusion

## References